In [2]:
#Luca: We must tile all the WSI before proceeding with either segmentation or WSI-level. This notebook does the preprocessing 


#Segmentation Pipeline
    # Tile
    # Give tile name based on coordinate
    # Extract polygon annotation
    # Generate Masks based on Polygon Annotation
    # Find BG using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)
    # Separate tiles into BG, Inf and Healthy folders
#WSI-level Pipeline
    # Tile
    # Remove BG tiles using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)
        # and name them sequentially

In [3]:
import os
#import os.path
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import pyvips as Vips
from tqdm import tqdm
from utils import vips_utils, normalize
from torchvision import transforms, utils
import time
import torchvision.models as models
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from PIL import Image, ImageFile
import statistics
from typing import Optional, Tuple
import pylibczi
from pylibczi import CziScene
import czifile
from czifile import CziFile 
import xml.etree.ElementTree as ET
import argparse
import gc 
import psutil
import resource
import platform
import pickle
import xmltodict
import time
import matplotlib.path as mpath
from skimage.draw import polygon
import cv2
import copy
import shutil

import torch
import random
import torchvision
import torch.nn as nn
from skimage import io, transform
from skimage.morphology import convex_hull_image

### PARAMETERS

In [4]:
TILE_SIZE = 512

WSI_DIR = '/cache/Inf_May23_dataset/'
SAVE_DIR = '/cache/S23_Infarct/patched_'+str(TILE_SIZE)+'/'
CZ_DIR = '/cache/S23_Infarct/annotation/'
MASK_DIR = '/cache/S23_Infarct/masks/'

SEGMENTATION_TILE_DIR = '/cache/S23_Infarct/twice_parsed_seg_data_'+str(TILE_SIZE)+'/'
WSI_TILE_DIR = '/cache/S23_Infarct/wsi_level_data_'+str(TILE_SIZE)+'/'

tile_czi = True
tile_svs = True

In [5]:
if not os.path.exists(WSI_DIR):
    print("WSI folder does not exist, script should stop now")
else:
    if not os.path.exists(SEGMENTATION_TILE_DIR):
        print("Tile folder you provided us does not exist, being created now...")
        os.makedirs(SEGMENTATION_TILE_DIR)
        
    if not os.path.exists(WSI_TILE_DIR):
        print("Tile folder for WSI-level you provided us does not exist, being created now...")
        os.makedirs(WSI_TILE_DIR)

    if not os.path.exists(SAVE_DIR):
        print("Tile folder you provided us does not exist, being created now...")
        os.makedirs(SAVE_DIR)

    if not os.path.exists(CZ_DIR):
        print("Annotation folder you provided us does not exist, being created now...")
        os.makedirs(CZ_DIR)

    if not os.path.exists(MASK_DIR):
        print("Mask folder you provided us does not exist, being created now...")
        os.makedirs(MASK_DIR)

    print("Found WSI folder... ")
    wsi_slides = os.listdir(WSI_DIR)
    imagenames = sorted(wsi_slides)
    print("All WSIs in wsi_dir: ")
    print(imagenames)

Found WSI folder... 
All WSIs in wsi_dir: 
['NA-5029-16_HE.svs', 'NA-5029-17_HE.svs', 'NA-5029-18_HE.svs', 'NA5004-16_HE.svs', 'NA5004-17_HE.svs', 'NA5004-18_HE.svs', 'NA5007-16_HE.svs', 'NA5007-17_HE.svs', 'NA5007-18_HE.svs', 'NA5009-16_HE.svs', 'NA5009-17_HE.svs', 'NA5009-18_HE.svs', 'NA5031-16_HE.svs', 'NA5031-17_HE.svs', 'NA5031-18_HE.svs', 'NA5041-16_HE.svs', 'NA5041-17_HE.svs', 'NA5041-18_HE.svs', 'NA5045-16_HE.svs', 'NA5045-17_HE.svs', 'NA5045-18_HE.svs', 'NA5051-16_HE.czi', 'NA5051-17_HE.czi', 'NA5051-18_HE.czi', 'NA5057-16_HE.svs', 'NA5057-17_HE.svs', 'NA5057-18_HE.svs', 'NA5063-16_HE.czi', 'NA5063-17_HE.czi', 'NA5063-18_HE.czi', 'NA5077-16_HE.czi', 'NA5077-17_HE.czi', 'NA5077-18_HE.czi', 'NA5085-16_HE.czi', 'NA5085-17_HE.czi', 'NA5085-18_HE.czi', 'NA5089-16_HE.czi', 'NA5089-17_HE.czi', 'NA5089-18_HE.czi', 'NA5090-16_HE.czi', 'NA5090-17_HE.czi', 'NA5090-18_HE.czi', 'NA5091-16_HE.czi', 'NA5091-17_HE.czi', 'NA5091-18_HE.czi', 'NA5093-16_HE.czi', 'NA5093-17_HE.czi', 'NA5093-18_HE

In [6]:
def grabCZI(path, verbose = False):
    img = czifile.imread(path)
    if verbose:
        print(img.shape)
        print(img)
    
    img = np.array(img, dtype = np.uint8)
    
    scenes = img.shape[0]
    time = img.shape[1]
    height = img.shape[2]
    width = img.shape[3]
    channels = img.shape[4]
    
    
    img = img.reshape((height, width, channels))
    if verbose:
        print(img)
        print(img.shape) 
        
    dtype_to_format = {
        'uint8': 'uchar',
        'int8': 'char',
        'uint16': 'ushort',
        'int16': 'short',
        'uint32': 'uint',
        'int32': 'int',
        'float32': 'float',
        'float64': 'double',
        'complex64': 'complex',
        'complex128': 'dpcomplex',
    }
    
    ###codes from numpy2vips
    height, width, bands = img.shape
    img = img.reshape(width * height * bands)
    vips = Vips.Image.new_from_memory(img.data, width, height, bands,
                                      dtype_to_format['uint8'])
    try: 
        del img, height, width, bands
        gc.collect()
    except: 
        pass
    
    return vips

In [6]:
###### SEGMENTATION / WSI-level Pipeline - Tile

print("Starting tiling....")
for imagename in tqdm(imagenames[:]):
    start = time.time()
    if imagename.split('.')[-1] == 'svs':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        vips_img = Vips.Image.new_from_file(WSI_DIR + imagename, level=0)
        
        print("Pre resize: ", vips_img.height, "x", vips_img.width)
        
        if NAID in ['NA5009-16_HE', 'NA5009-17_HE', 'NA5009-18_HE']:
            pass
            #vips_img = vips_img.resize(0.25)
        else:
            print("Resizing small case")
            vips_img = vips_img.resize(2)
            
        print("Post resize: ", vips_img.height, "x", vips_img.width)
            
        print("Loaded Image: " + WSI_DIR + imagename)

        if tile_svs:
        
            vips_utils.save_and_tile(vips_img, os.path.splitext(imagename)[0] \
                                     , SAVE_DIR, tile_size = TILE_SIZE)
            print("Done Tiling: ", WSI_DIR + imagename)
        
    elif imagename.split('.')[-1] == 'czi':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        try: 
            vips_img = grabCZI(WSI_DIR + imagename)
            print("Loaded Image: " + WSI_DIR + imagename)
            
            print("Pre resize: ", vips_img.height, "x", vips_img.width)
            
            if tile_czi:
        
                vips_utils.save_and_tile(vips_img, os.path.splitext(imagename)[0] \
                                         , SAVE_DIR, tile_size = TILE_SIZE)

                print("Done Tiling: ", WSI_DIR + imagename)
                del vips_img
                gc.collect()
                print("Finish Delete", WSI_DIR + imagename)
        except:
            print("Error in tiling")
        
        

    else:
        print("Skipped,", imagename, '. This file is either not .czi or .svs, or not the file assigned')
    
    try: 
        del vips_img 
        gc.collect()
    except:
        pass
    
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

Starting tiling....


  0%|          | 0/69 [00:00<?, ?it/s]

Loading NA-5029-16_HE.svs  ......
Pre resize:  34033 x 45816
Resizing small case
Post resize:  68066 x 91632
Loaded Image: /cache/Inf_May23_dataset/NA-5029-16_HE.svs
/cache/S23_Infarct/patched_512/NA-5029-16_HE


  1%|▏         | 1/69 [01:12<1:22:37, 72.91s/it]

Done Tiling:  /cache/Inf_May23_dataset/NA-5029-16_HE.svs
processed in  72.91144609451294  seconds
____________________________________________
Loading NA-5029-17_HE.svs  ......
Pre resize:  33509 x 53784
Resizing small case
Post resize:  67018 x 107568
Loaded Image: /cache/Inf_May23_dataset/NA-5029-17_HE.svs
/cache/S23_Infarct/patched_512/NA-5029-17_HE


...

Done Tiling:  /cache/Inf_May23_dataset/NA5161-16_HE.czi
Finish Delete /cache/Inf_May23_dataset/NA5161-16_HE.czi
processed in  80.53560066223145  seconds
____________________________________________
Loading NA5161-17_HE.czi  ......
Loaded Image: /cache/Inf_May23_dataset/NA5161-17_HE.czi
Pre resize:  87597 x 105358
/cache/S23_Infarct/patched_512/NA5161-17_HE


 99%|█████████▊| 68/69 [1:32:23<01:20, 80.24s/it]

Done Tiling:  /cache/Inf_May23_dataset/NA5161-17_HE.czi
Finish Delete /cache/Inf_May23_dataset/NA5161-17_HE.czi
processed in  102.05071091651917  seconds
____________________________________________
Loading NA5161-18_HE.czi  ......
Loaded Image: /cache/Inf_May23_dataset/NA5161-18_HE.czi
Pre resize:  63797 x 70783
/cache/S23_Infarct/patched_512/NA5161-18_HE


100%|██████████| 69/69 [1:33:13<00:00, 81.07s/it]

Done Tiling:  /cache/Inf_May23_dataset/NA5161-18_HE.czi
Finish Delete /cache/Inf_May23_dataset/NA5161-18_HE.czi
processed in  50.153114318847656  seconds
____________________________________________


In [7]:
###### SEGMENTATION - Give tile name based on coordinate

print("About to change names to add coordinates")
for case_folder in sorted(os.listdir(SAVE_DIR)):
    NAID = case_folder
    print('Processing NAID: ', NAID)
    for tile_folder in sorted(os.listdir(SAVE_DIR+NAID+'/0/')):
        # folder_level == y axis distance determinant
        # file_level == x axis distance determinant
        y0 = int(tile_folder)*TILE_SIZE
        y1 = (int(tile_folder)+1)*TILE_SIZE
        #print("Y axis = (", y0,',',y1,')')
        for tile_file in sorted(os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder+'/')):
            x0 = int(tile_file.split('.')[0])*TILE_SIZE
            x1 = (int(tile_file.split('.')[0])+1)*TILE_SIZE
            #print("X axis = (", x0,',',x1,')')
            #print("Renaming ", SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+tile_file)
            os.rename(SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+tile_file, 
                      SAVE_DIR+NAID+'/0/'+str(tile_folder)+'/'+str(y0)+'_'+str(x0)+'_'+str(y1)+'_'+str(x1)+'_'+tile_file)
            #print("Renamed into ", SAVE_DIR+NAID+'/0/'+tile_folder+'/'+str(y0)+'_'+str(x0)+'_'+str(y1)+'_'+str(x1)+'_'+t

About to change names to add coordinates
Processing NAID:  NA-5029-16_HE
Processing NAID:  NA-5029-17_HE
Processing NAID:  NA-5029-18_HE
Processing NAID:  NA5004-16_HE
Processing NAID:  NA5004-17_HE
Processing NAID:  NA5004-18_HE
Processing NAID:  NA5007-16_HE
Processing NAID:  NA5007-17_HE
Processing NAID:  NA5007-18_HE
Processing NAID:  NA5009-16_HE
Processing NAID:  NA5009-17_HE
Processing NAID:  NA5009-18_HE
Processing NAID:  NA5031-16_HE
Processing NAID:  NA5031-17_HE
Processing NAID:  NA5031-18_HE
Processing NAID:  NA5041-16_HE
Processing NAID:  NA5041-17_HE
Processing NAID:  NA5041-18_HE
Processing NAID:  NA5045-16_HE
Processing NAID:  NA5045-17_HE
Processing NAID:  NA5045-18_HE
Processing NAID:  NA5051-16_HE
Processing NAID:  NA5051-17_HE
Processing NAID:  NA5051-18_HE
Processing NAID:  NA5057-16_HE
Processing NAID:  NA5057-17_HE
Processing NAID:  NA5057-18_HE
Processing NAID:  NA5063-16_HE
Processing NAID:  NA5063-17_HE
Processing NAID:  NA5063-18_HE
Processing NAID:  NA5077-1

In [8]:
###### SEGMENTATION - Extract polygon annotation

for imagename in tqdm(imagenames[:]):
    start = time.time()
    if imagename.split('.')[-1] == 'svs':
        pass
        
    elif imagename.split('.')[-1] == 'czi':
        NAID = imagename.split('.')[0]
        print("Loading", imagename, " ......")
        try: 
            czifile = pylibczi.CziFile(WSI_DIR + imagename, metafile_out = CZ_DIR + NAID + '.cz',use_pylibczi=True, verbose=True)
            czifile.read_meta()
            
            tree = ET.parse(CZ_DIR + NAID + '.cz') 
            root = tree.getroot() 
            
            tree.write(CZ_DIR + NAID + '.xml')
        except:
            print("Error in extracting annotation")
            
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

  0%|          | 0/69 [00:00<?, ?it/s]

processed in  1.9073486328125e-06  seconds
____________________________________________
processed in  9.5367431640625e-07  seconds
____________________________________________
processed in  9.5367431640625e-07  seconds
____________________________________________
processed in  1.1920928955078125e-06  seconds
____________________________________________
processed in  9.5367431640625e-07  seconds
____________________________________________
processed in  7.152557373046875e-07  seconds
____________________________________________
processed in  4.76837158203125e-07  seconds
____________________________________________
processed in  4.76837158203125e-07  seconds
____________________________________________
processed in  7.152557373046875e-07  seconds
____________________________________________
processed in  4.76837158203125e-07  seconds
____________________________________________
processed in  4.76837158203125e-07  seconds
____________________________________________
processed in  4.76837

 42%|████▏     | 29/69 [00:00<00:00, 56.55it/s]

processed in  0.31695127487182617  seconds
____________________________________________
Loading NA5051-17_HE.czi  ......
processed in  0.04110527038574219  seconds
____________________________________________
Loading NA5051-18_HE.czi  ......
processed in  0.03639864921569824  seconds
____________________________________________
processed in  2.6226043701171875e-06  seconds
____________________________________________
processed in  1.1920928955078125e-06  seconds
____________________________________________
processed in  1.1920928955078125e-06  seconds
____________________________________________
Loading NA5063-16_HE.czi  ......
processed in  0.04552507400512695  seconds
____________________________________________
Loading NA5063-17_HE.czi  ......
processed in  0.04552483558654785  seconds
____________________________________________
Loading NA5063-18_HE.czi  ......
processed in  0.043135881423950195  seconds
____________________________________________
Loading NA5077-16_HE.czi  ......


 51%|█████     | 35/69 [00:00<00:00, 34.86it/s]

...

processed in  0.04732966423034668  seconds
____________________________________________
Loading NA5114-18_HE.czi  ......
processed in  0.047271728515625  seconds
____________________________________________
Loading NA5116-16_HE.czi  ......
processed in  0.03992033004760742  seconds
____________________________________________
Loading NA5116-17_HE.czi  ......
processed in  0.04051804542541504  seconds
____________________________________________
Loading NA5116-18_HE.czi  ......


 97%|█████████▋| 67/69 [00:02<00:00, 25.21it/s]

processed in  0.12010645866394043  seconds
____________________________________________
Loading NA5137-16_HE.czi  ......
processed in  0.0431218147277832  seconds
____________________________________________
Loading NA5137-17_HE.czi  ......
processed in  0.04361104965209961  seconds
____________________________________________
Loading NA5137-18_HE.czi  ......
processed in  0.04049277305603027  seconds
____________________________________________
processed in  5.4836273193359375e-06  seconds
____________________________________________
processed in  2.1457672119140625e-06  seconds
____________________________________________
processed in  1.6689300537109375e-06  seconds
____________________________________________
Loading NA5161-16_HE.czi  ......
processed in  0.046466827392578125  seconds
____________________________________________
Loading NA5161-17_HE.czi  ......


100%|██████████| 69/69 [00:02<00:00, 28.05it/s]

processed in  0.04495358467102051  seconds
____________________________________________
Loading NA5161-18_HE.czi  ......
processed in  0.0419158935546875  seconds
____________________________________________


In [9]:
def grab_svs_annot(path):
    doc = xmltodict.parse(open(path, 'r', encoding='utf-8').read())
    
    all_annot = []
    if type(doc['Annotations']['Annotation']['Regions']['Region']) == list:
        for region_idx in range(len(doc['Annotations']['Annotation']['Regions']['Region'])):
            coords = []
            for annot in doc['Annotations']['Annotation']['Regions']['Region'][region_idx]['Vertices']['Vertex']:
                coord = [int(annot['@Y']),int(annot['@X'])]

                coords.append(coord)
            all_annot.append(coords)
    else:
        coords = []
        for annot in doc['Annotations']['Annotation']['Regions']['Region']['Vertices']['Vertex']:
            coord = [int(annot['@Y']),int(annot['@X'])]

            coords.append(coord)
        all_annot.append(coords)
    
    return all_annot

#from: https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-dictionaries-and-lists
def findkeys(node, kv):
    if isinstance(node, list):
        for i in node:
            for x in findkeys(i, kv):
               yield x
    elif isinstance(node, dict):
        if kv in node:
            yield node[kv]
        for j in node.values():
            for x in findkeys(j, kv):
                yield x

def grab_czi_annot(path):
    doc = xmltodict.parse(open(path, 'r', encoding='utf-8').read())
    
    amt_annot = len(list(findkeys(doc,'Points')))
    
    all_annot = []
    coords = []
    
    #print("AMOUNT ANNOT = ", amt_annot)
    
    for annot_idx in range(amt_annot):
        coords = []
        all_cord = list(findkeys(doc,'Points'))[annot_idx].split(' ')
        isGood = False
        for xy in all_cord:
            #print(xy)
            if int(float(xy.split(',')[1])) < 0 or int(float(xy.split(',')[0])) < 0:
                pass
                #print("Passed on negative coordinates")
            else:
                coord = [int(float(xy.split(',')[1])),int(float(xy.split(',')[0]))]
                isGood = True
                
                coords.append(coord)
        if isGood:
            all_annot.append(coords)
    
    return all_annot



def create_polygon_mask(vertex_coordinates, mask):
    
    rr, cc = polygon(vertex_coordinates[:,0],vertex_coordinates[:,1],mask.shape)
    mask[rr,cc] = 1
    
    return mask


# https://stackoverflow.com/questions/41925853/fill-shapes-contours-using-numpy
def fill_contours(arr):
    return np.maximum.accumulate(arr,1) & \
           np.maximum.accumulate(arr[:,::-1],1)[:,::-1]

In [13]:
###### SEGMENTATION - Generate Masks based on Polygon Annotation

gt_df = pd.read_csv('/cache/S23_Infarct/gt.csv')

for imagename in tqdm(imagenames[:]):
    start = time.time()

    wsi_gt = gt_df[gt_df.cases == imagename].iat[0,2]
    
    resized = False
    if wsi_gt == 1:
        if imagename.split('.')[-1] == 'svs':
            NAID = imagename.split('.')[0]
            print("Loading", imagename, " ......")
            if True: 
                vips_img = Vips.Image.new_from_file(WSI_DIR + imagename, level=0)
                print("Loaded Image: " + WSI_DIR + imagename)
                
                if NAID in ['NA5009-16_HE', 'NA5009-17_HE', 'NA5009-18_HE']:
                    pass
                    #vips_img = vips_img.resize(0.25)
                else:
                    print("Resizing small case")
                    vips_img = vips_img.resize(2)
                    resized = True

                dimension = [vips_img.height, vips_img.width]

                all_annot = grab_svs_annot(CZ_DIR + NAID + '.xml')

                mask = np.zeros(dimension,'uint8')
                
                # It is all matching xml up to here
                #print(all_annot)
                
                print("Starting to build mask")
                print("Number of annotation groups =", len(all_annot))
                for coords in all_annot:
                    #Old method - faulty
                    #mask = create_polygon_mask(coords, mask)
                    
                    #New method
                    max_y = 0
                    max_x = 0
                    min_y = 10000000000
                    min_x = 10000000000
                    
                    submask = np.zeros(dimension,'uint8')
                    for coord in coords:
                        y = coord[0]
                        x = coord[1]
                        
                        #print("y = ", y, "x = ", x)
                        if resized:
                            y = y*2
                            x = x*2
                            submask[y][x] = 1
                        else:
                            submask[y][x] = 1
                            
                        if y > max_y:
                            max_y = y
                            
                        if x > max_x:
                            max_x = x
                            
                        if y < min_y:
                            min_y = y
                            
                        if x < min_x:
                            min_x = x
                    
                    submask[min_y:max_y+1, min_x:max_x+1] = convex_hull_image(submask[min_y:max_y+1, min_x:max_x+1])
                    
                    mask = mask + submask
                        
                
                #print("Filling contours")
                #mask = fill_contours(mask)

                #mask = np.packbits(mask,axis=None)
                #print("Binarized mask")
                
                print("Saving mask contours")
                np.save(MASK_DIR+NAID+'.npy',mask)

                print("processed in ", time.time()-start," seconds")
                print("____________________________________________")

            else:
                print("Error in generating masks from polygon for ", NAID)


        elif imagename.split('.')[-1] == 'czi':
            NAID = imagename.split('.')[0]
            print("Loading", imagename, " ......")
            if True: 
                vips_img = grabCZI(WSI_DIR + imagename)
                print("Loaded Image: " + WSI_DIR + imagename)

                dimension = [vips_img.height, vips_img.width]
                all_annot = grab_czi_annot(CZ_DIR + NAID + '.xml')

                mask = np.zeros(dimension,'uint8')
                
                # It is all matching up to here
                #print(all_annot)
                
                print("Starting to build mask")
                print("Number of annotation groups =", len(all_annot))
                for coords in all_annot:
                    #Old method - faulty
                    #mask = create_polygon_mask(coords, mask)
                    
                    #New method
                    max_y = 0
                    max_x = 0
                    min_y = 10000000000
                    min_x = 10000000000
                    
                    submask = np.zeros(dimension,'uint8')
                    for coord in coords:
                        y = coord[0]
                        x = coord[1]
                        
                        #print("y = ", y, "x = ", x)
                        if resized:
                            y = y*2
                            x = x*2
                            submask[y][x] = 1
                        else:
                            submask[y][x] = 1
                            
                        if y > max_y:
                            max_y = y
                            
                        if x > max_x:
                            max_x = x
                            
                        if y < min_y:
                            min_y = y
                            
                        if x < min_x:
                            min_x = x
                    
                    submask[min_y:max_y+1, min_x:max_x+1] = convex_hull_image(submask[min_y:max_y+1, min_x:max_x+1])
                    
                    mask = mask + submask
                        
                #print("Filling contours")
                #mask = fill_contours(mask)
                

                #mask = np.packbits(mask,axis=None)
                #print("Binarized mask")
                
                print("Saving mask contours")
                np.save(MASK_DIR+NAID+'.npy',mask) 

                print("processed in ", time.time()-start," seconds")
                print("____________________________________________")
            else:
                print("Error in generating masks from polygon for ", NAID)
        
        
    else:
        print("Skipped,", imagename, '. This file is either not .czi or .svs, or not the file assigned')
    
    if wsi_gt == 1:
        try:
            del vips_img,coords,mask,submask
            gc.collect()
        except:
            pass

  0%|          | 0/69 [00:00<?, ?it/s]

Skipped, NA-5029-16_HE.svs . This file is either not .czi or .svs, or not the file assigned
Skipped, NA-5029-17_HE.svs . This file is either not .czi or .svs, or not the file assigned
Loading NA-5029-18_HE.svs  ......
Loaded Image: /cache/Inf_May23_dataset/NA-5029-18_HE.svs
Resizing small case
Starting to build mask
Number of annotation groups = 9
Saving mask contours


  4%|▍         | 3/69 [01:50<40:40, 36.97s/it]

processed in  110.78698921203613  seconds
____________________________________________
Skipped, NA5004-16_HE.svs . This file is either not .czi or .svs, or not the file assigned
Skipped, NA5004-17_HE.svs . This file is either not .czi or .svs, or not the file assigned
Loading NA5004-18_HE.svs  ......
Loaded Image: /cache/Inf_May23_dataset/NA5004-18_HE.svs
Resizing small case
Starting to build mask
Number of annotation groups = 1
Saving mask contours


  9%|▊         | 6/69 [01:55<16:54, 16.10s/it]

...

processed in  41.64433288574219  seconds
____________________________________________
Loading NA5146-18_HE.svs  ......
Loaded Image: /cache/Inf_May23_dataset/NA5146-18_HE.svs
Resizing small case
Starting to build mask
Number of annotation groups = 1
Saving mask contours


 96%|█████████▌| 66/69 [33:46<02:00, 40.21s/it]

processed in  11.84977650642395  seconds
____________________________________________
Skipped, NA5161-16_HE.czi . This file is either not .czi or .svs, or not the file assigned
Loading NA5161-17_HE.czi  ......
Loaded Image: /cache/Inf_May23_dataset/NA5161-17_HE.czi
Starting to build mask
Number of annotation groups = 2
Saving mask contours
processed in  67.65470099449158  seconds
____________________________________________


100%|██████████| 69/69 [34:54<00:00, 30.35s/it]

Skipped, NA5161-18_HE.czi . This file is either not .czi or .svs, or not the file assigned


In [14]:
###### SEGMENTATION - Find BG using ../saved_models/patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt (in 224)

MODEL_SEG_DIR = './patched_512_NewAnnotation+preprocess_ResNet18_Feb12.pt'

seg_model = torchvision.models.resnet18()
seg_model.fc = nn.Linear(512, 3)

checkpoint = torch.load(MODEL_SEG_DIR)
seg_model.load_state_dict(checkpoint['model_state_dict'])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
seg_model.to(device)

# # Brainseg
# BS_SEG_DIR = './ResNet18_19.pkl'
# brainsec_model = torch.load(BS_SEG_DIR, map_location=lambda storage, loc: storage)
# brainsec_model = brainsec_model.cuda()
    


for NAID in os.listdir(SAVE_DIR):
    start = time.time()
    print("Processing ", NAID)
    
    try:
        os.makedirs(SEGMENTATION_TILE_DIR+NAID)
    except:
        pass
    
    try:
        os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/BG/')
    except:
        pass
    
    for tile_folder in os.listdir(SAVE_DIR+NAID+'/0/'):
        for tile in os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder):
            seg_model.train(False)
            brainsec_model.train(False)
            
            img = Image.open(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile)
            
            img_tensor = transforms.ToTensor()(img)
            img_tensor = transforms.Normalize(mean=[0.4409763317567454, 0.4016568471536302, 0.4988298669112181],
                             std=[0.31297803931100737, 0.2990562933047881, 0.33747493782548915])(img_tensor)
            img_tensor = torch.reshape(img_tensor,(1,3,512,512))
            img_tensor = img_tensor.cuda()
            
            predict = seg_model(img_tensor)
            preds = F.sigmoid(predict)
            _, indices = torch.max(predict.data, 1) # indices = 0:Background, 1:WM, 2:GM
            indices = indices.type(torch.uint8)
            running_seg =  indices.data.cpu()
            
#             predict2 = brainsec_model(img_tensor)
#             preds2 = F.sigmoid(predict2)
#             #print(preds)
#             _, indices2 = torch.max(predict2.data, 1) # indices = 0:Background, 1:WM, 2:GM
#             indices2 = indices.type(torch.uint8)
#             running_seg2 =  indices.data.cpu()
            
            
            if running_seg == 0:# or running_seg2 == 0:
                shutil.move(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/BG/'+tile)
            else:
                pass
                #print("bg removed")
                
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

/home/lzhengfe/anaconda3/envs/vips/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [8]:
###### SEGMENTATION - Separate tiles into BG, Inf and Healthy folders
gt_df = pd.read_csv('/cache/S23_Infarct/gt.csv')

for imagename in tqdm(imagenames[:]):
    start = time.time()

    wsi_gt = gt_df[gt_df.cases == imagename].iat[0,2]
    
    NAID = imagename.split('.')[0]
    print("Loading", imagename, " ......")

    try:
        os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/Heal/')
    except:
        pass
    
    if wsi_gt == 1:
        try:
            os.makedirs(SEGMENTATION_TILE_DIR+NAID+'/Inf/')
        except:
            pass
        mask = np.load(MASK_DIR+NAID+'.npy')
    
        print("Loaded mask at ", MASK_DIR+NAID+'.npy' ,", about to process...")

    #mask = np.unpackbits(mask,count=im_size).reshape(img_dim).view(bool)
    #mask = mask.view(np.uint8)
    #print("Unbinarized mask")

    for tile_folder in os.listdir(SAVE_DIR+NAID+'/0/'):
        for tile in os.listdir(SAVE_DIR+NAID+'/0/'+tile_folder):
            
            try:
                #file naming convention --> y0 x0 y1 x1
                y0 = int(tile.split('_')[0])
                x0 = int(tile.split('_')[1])
                y1 = int(tile.split('_')[2])
                x1 = int(tile.split('_')[3])

                if wsi_gt == 1:
                    if sum(sum(mask[y0:y1,x0:x1])) >= ((512*512)/2):
                        shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/Inf/'+tile)
                    elif sum(sum(mask[y0:y1,x0:x1])) == 0:
                        shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/Heal/'+tile)
                    else:
                        pass
                else:
                    shutil.copyfile(SAVE_DIR+NAID+'/0/'+tile_folder+'/'+tile, SEGMENTATION_TILE_DIR+NAID+'/Heal/'+tile)
            except:
                print("Issue with tile ", tile)
                
    print("processed in ", time.time()-start," seconds")
    print("____________________________________________")

  0%|          | 0/69 [00:00<?, ?it/s]

Loading NA-5029-16_HE.svs  ......


  1%|▏         | 1/69 [00:11<13:20, 11.78s/it]

processed in  11.777730226516724  seconds
____________________________________________
Loading NA-5029-17_HE.svs  ......


  3%|▎         | 2/69 [00:30<17:56, 16.07s/it]

...

processed in  3.309877395629883  seconds
____________________________________________
Loading NA5161-17_HE.czi  ......
Loaded mask at  /cache/S23_Infarct/masks/NA5161-17_HE.npy , about to process...


 99%|█████████▊| 68/69 [21:01<00:23, 23.68s/it]

processed in  45.86230278015137  seconds
____________________________________________
Loading NA5161-18_HE.czi  ......


100%|██████████| 69/69 [21:02<00:00, 18.30s/it]

processed in  1.986387014389038  seconds
____________________________________________
